# [CSV exercise: storm data]()

[source](https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/)



In [ ]:
ls sources/

[sources/StormEvents_details-ftp_v1.0_d2017_c20180918.csv](sources/StormEvents_details-ftp_v1.0_d2017_c20180918.csv)

In [ ]:
import pandas as pd

In [ ]:
storms = pd.read_csv('sources/StormEvents_details-ftp_v1.0_d2017_c20180918.csv')  # Needs an argument...
storms.head()

In [ ]:
storms.iloc[0]

# Plain-python evaluation

## Looping

In [ ]:
for row in list(storms.itertuples())[:10]:
    print(row.STATE)

## Gather some statistics...

Ideas for the state you were born in...
- How many events of each type?
- Sum of property damage?
- How many events, by month?

In [ ]:
n_events = 0 
for row in storms.itertuples():
    if row.STATE == 'MINNESOTA':  # your filter here
        n_events += 1  # please do something more interesting
print(f'{n_events} events total')

# Applying Pandas

In [ ]:
storms.EVENT_TYPE.value_counts()

# Data cleanup

In [ ]:
storms.DAMAGE_CROPS.head()

In [ ]:
def str_to_num(x):
    abbrevs = {'K': 1000, 'M': 1000000, 'B': 1000000000}
    try:
        suffix = x.strip()[-1]
    except AttributeError:  # not a number at all 
        return x
    if suffix in abbrevs:
        multiplier = abbrevs[suffix]
        quant = float(x.strip()[:-1])
    else:
        multiplier = 1 
        quant = float(x.strip())
    return quant * multiplier


In [ ]:
str_to_num('3.4K')

In [ ]:
str_to_num('360 ')

In [ ]:
str_to_num(np.NaN)

In [ ]:
storms['damage_crops_cleaned'] = storms.DAMAGE_CROPS.apply(str_to_num)

In [ ]:
storms[['DAMAGE_CROPS', 'damage_crops_cleaned']].head()

# Filter

In [ ]:
storms[storms.STATE == 'MINNESOTA']

Can you figure out (or look up) filtering by more than one criterion?

## Ideas

- Most costly event?
- Average $ damage, by event type?
- Most common event by state?

## Pass your own functions

Here's a boring example.  Can you make a better one?

In [ ]:
storms.DAMAGE_PROPERTY.head()

In [ ]:
# Remember how to apply str_to_num?
storms['damage_property_cleaned'] = storms.DAMAGE_PROPERTY.apply(str_to_num)

In [ ]:
import numpy as np
def crops_to_property(storm):
    "Ratio of crop damage to property damage."
    
    return np.divide(storm.damage_crops_cleaned, storm.damage_property_cleaned)

In [ ]:
storms['crops_to_property'] = storms.apply(crops_to_property, axis=1)

# Format fixes

In [ ]:
storms.BEGIN_DATE_TIME.head()

In [ ]:
pd.to_datetime(storms.BEGIN_DATE_TIME).head()

In [ ]:
storms['begin'] = pd.to_datetime(storms.BEGIN_DATE_TIME)

# Make a filtered subset

In [ ]:
storms[storms.damage_crops_cleaned > 0 &
       storms.damage_crops_cleaned.notnull()] \
       .damage_crops_cleaned.head()

In [ ]:
damaging_storms = storms[storms.damage_crops_cleaned > 0 &
       storms.damage_crops_cleaned.notnull()]

# Plot with [Altair](https://altair-viz.github.io/)

In [ ]:
import altair as alt

Altair requires some extra work for [large data](https://altair-viz.github.io/user_guide/faq.html)

In [ ]:
chart = alt.Chart(damaging_storms).mark_bar().encode(
        alt.X('damage_property_cleaned:Q', bin=True),
        y='count()')

with alt.data_transformers.enable('json'):
    chart.display()

In [ ]:
chart = alt.Chart(damaging_storms).mark_point().encode(
        x='begin', y='damage_property_cleaned')

with alt.data_transformers.enable('json'):
    chart.display()

In [ ]:
chart = alt.Chart(damaging_storms).mark_point().encode(
        x='begin:T', 
        y=alt.Y('damage_property_cleaned:Q', 
                   scale=alt.Scale(type='log'))
)

with alt.data_transformers.enable('json'):
    chart.display()

In [ ]:
damaging_storms.BEGIN_DATE_TIME.head()

In [ ]:
chart = altair.Chart(damaging_storms).mark_point().encode(
        altair.X('damage_property_cleaned:Q', bin=True),
        y='count()')

with altair.data_transformers.enable('json'):
    chart.display()

# Function on DataFrame

In [ ]:
talks['first_last_ratio'] = talks.apply(
    lambda talk: np.divide(len(talk.firstname), len(talk.lastname)),
    axis=1
    )


In [ ]:
talks[['firstname', 'lastname', 'first_last_ratio']]

In [ ]:
talks[talks.first_last_ratio == talks.first_last_ratio.max()]

In [ ]:
talks.loc[talks.first_last_ratio.idxmax()]